In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from dataset import extract_users_movies_ratings_lists, TripletDataset
import torch 

#Useful constants
number_of_users, number_of_movies = (10000, 1000)
RANDOM_STATE = 58
DATA_DIR = '../data'


data_pd = pd.read_csv(DATA_DIR+'/data_train.csv')[:100]
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)




In [3]:
import time
import pandas as pd
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import SVD, SVDpp, NCF, ALS, UserCF, ItemCF, RNN4Rec

# remove unnecessary tensorflow logging
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_WARNINGS"] = "FALSE"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2022-07-15 14:44:39.525323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-15 14:44:39.525352: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})

In [5]:
train_data, data_info = DatasetPure.build_trainset(train)
eval_data = DatasetPure.build_evalset(val)

In [14]:
with tf.compat.v1.variable_scope('test'):
    svdpp = SVDpp(task="rating", data_info=data_info, embed_size=16,
                    n_epochs=1, lr=0.001, reg=None, batch_size=256)
    svdpp.fit(train_data, verbose=2, eval_data=eval_data,
                metrics=["rmse", "mae", "r2"])

Training start time: 2022-07-15 14:54:13


ValueError: Variable test/bu_var already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/gio/.local/lib/python3.9/site-packages/libreco/algorithms/svdpp.py", line 80, in _build_model
    self.bu_var = tf.get_variable(name="bu_var", shape=[self.n_users],
  File "/home/gio/.local/lib/python3.9/site-packages/libreco/algorithms/svdpp.py", line 145, in fit
    self._build_model(sparse_implicit_interaction)
  File "/tmp/ipykernel_44116/3365660772.py", line 4, in <module>
    svdpp.fit(train_data, verbose=2, eval_data=eval_data,
  File "/home/gio/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/gio/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3364, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):


(1176952,)

In [ ]:
# ncf = NCF("rating", data_info, embed_size=16, n_epochs=10, lr=0.001,
#             lr_decay=False, reg=None, batch_size=256, num_neg=1, use_bn=True,
#             dropout_rate=None, hidden_units="128,64,32", tf_sess_config=None)
# ncf.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
#         metrics=["rmse", "mae", "r2"])

In [25]:
import scipy
def combine_models(mu, sigma, yhat):
    coeff = np.linspace(10, 40+1)
    coeff = scipy.stats.norm.pdf(coeff, loc=mu, scale=sigma)
    coeff = coeff / coeff.sum()
    print(coeff)
    print(coeff.sum())


In [26]:
combine_models(10, 5.0, [])

[9.61055658e-02 9.53393119e-02 9.30770122e-02 8.94251757e-02
 8.45520452e-02 7.86747514e-02 7.20432985e-02 6.49230247e-02
 5.75772408e-02 5.02516006e-02 4.31614372e-02 3.64828571e-02
 3.03479063e-02 2.48436598e-02 2.00147127e-02 1.58682887e-02
 1.23810585e-02 9.50675722e-03 7.18379579e-03 5.34222900e-03
 3.90965081e-03 2.81579081e-03 1.99576660e-03 1.39208599e-03
 9.55584991e-04 6.45534600e-04 4.29157471e-04 2.80776514e-04
 1.80780600e-04 1.14548608e-04 7.14290310e-05 4.38335502e-05
 2.64719225e-05 1.57329899e-05 9.20203741e-06 5.29667921e-06
 3.00033878e-06 1.67256832e-06 9.17580942e-07 4.95395264e-07
 2.63212350e-07 1.37628261e-07 7.08200011e-08 3.58633771e-08
 1.78728325e-08 8.76561784e-09 4.23076232e-09 2.00956321e-09
 9.39359130e-10 4.32124209e-10]
1.0
